## Parsing of the dataset CORD-19


### Importing the parser and parsing the datasets

In [1]:
from Parser import *

In [2]:
#Creating a Parser and specifying what kind of dataset we want to parse
parser = Parser([Dataset.BIORXIV])
parser.parse(indexByFile = False);

### Example of accesing the paper by index

In [12]:
#We can access the date by index or by file name, but we have to change in the parse function
#what kind of invoke we want
print(parser.data_dicts[Dataset.BIORXIV][45])

Title
How domain growth is implemented determines the long term behaviour of a cell population through its effect on spatial correlations

Abstract
Domain growth plays an important role in many biological systems, and so the inclusion of domain growth in models of these biological systems is important to understanding how these biological systems function. In this work we present methods to include the effects of domain growth on the evolution of spatial correlations in a continuum approximation of a lattice-based model of cell motility and proliferation. We show that, depending on the way in which domain growth is implemented, different steady-state densities are predicted for an agent population. Furthermore, we demonstrate that the way in which domain growth is implemented can result in the evolution of the agent density depending on the size of the domain. Continuum approximations that ignore spatial correlations cannot capture these behaviours, while those that account for spatial

### Accesing certain elements of the paper

In [6]:
#By method titles(), abstracts() and bodies() you can access to certain elements of the paper
paper_abstracts = parser.abstracts()
for abstract in paper_abstracts[Dataset.BIORXIV].values():
    print(abstract)

Abstract
Summary Statement: DUX4 is thought to mediate cytopathology through p53. Here, DUX4 is shown to kill primary myoblasts and promote pathological phenotypes in the iDUX4[2.7] mouse model on the p53-null background, calling into question this notion.FSHD is a genetically dominant myopathy caused by mutations that cause expression of the normally silent DUX4 gene. This transcription factor has been shown to interfere with myogenesis when misexpressed at very low levels in myoblasts, and to cause cell death when overexpressed at high levels. A previous report using adeno-associated virus to deliver high levels of DUX4 to mouse skeletal muscle demonstrated severe pathology that was suppressed on a p53 knockout background, implying that DUX4 acted through the p53 pathway. Here, we investigate the p53-dependence of DUX4 using both in vitro cellular and the transgenic iDUX4[2.7] mouse models. We find that inhibiting p53 has no effect on the cytoxicity of DUX4 in vitro. When crossed ont